# Spark Caching

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import *
from pyspark.sql import functions as F

In [ ]:
spark = SparkSession \
  .builder \
  .appName("Caching") \
  .master("local[4]") \
  .getOrCreate()

## Подготовка данных

In [ ]:
!wget -O /tmp/taxi.parquet 'https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet'

In [ ]:
taxi_df = spark.read.parquet("/tmp/taxi.parquet")

In [ ]:
taxi_df.printSchema()

In [ ]:
taxi_df.rdd.getNumPartitions()

In [ ]:
taxi_df.groupby(F.spark_partition_id()).count().show()

# Особенности кэширования

Закэшируем запрос:

In [ ]:
taxi_cached_df = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 3) \
  .cache()

In [ ]:
taxi_cached_df.explain()

### Ситуация 1. Поменять операции местами

Рассмотрим запрос аналогичный предыдущему, в котром операции `filter` и `select` поменялись местами:

In [ ]:
taxi_select_after_filter_df = taxi_df.filter(taxi_df.passenger_count > 3) \
  .select(taxi_df.trip_distance, taxi_df.passenger_count)

**Вопрос:** удастся ли Spark воспользоваться кэшем?

In [ ]:
taxi_select_after_filter_df.explain()

<details>
    <summary><strong>Ответ</strong></summary>
    <p>Не удалось.</p>
    <p><i>Cache Manager</i> вступает в игру на стадии после логического плана, но перед оптимизатором. После оптимизатора планы запроса для `taxi_select_after_filter_df` и `taxi_cached_df` будут одинаковыми. Логические планы (Analyzed Logical Plans) у них разные, поэтому кэширование не используется в работе</p>
</details>

### Ситуация 2. Усилить условие

Рассмотрим запрос, аналогичный `taxi_cached_df`, в котором услвие усилено:

In [ ]:
taxi_stronger_condition_df = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 7)

**Вопрос**: удастся ли Spark воспользоваться кэшем?

In [ ]:
taxi_stronger_condition_df.explain()

<details>
    <summary><strong>Ответ</strong></summary>
    <p>Не удалось.</p>
    <p>Чисто логически можно заключить, что данные находятся в кэше, но Spark не будет их читать оттуда, потому что логические планы (Analyzed Logical Plans) отличаются: в этот раз условие фильтрации другое.</p>
    <p>Воспользоваться кэшем в этот раз можно следующим образом:</p>
</details>

In [ ]:
taxi_stronger_condition_df = taxi_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 3) \
  .filter(taxi_df.passenger_count > 7) \
  .explain()

### Ситуация 3. Выбрать меньше полей

Рассмотрим запрос, который выбирает меньше полей, чем `taxi_cached_df`: 

In [ ]:
taxi_fewer_columns_df = taxi_df.select(taxi_df.trip_distance) \
  .filter(taxi_df.passenger_count > 3)

**Вопрос:** удастся ли Spark воспользоваться кэшем?

In [ ]:
taxi_fewer_columns_df.explain()

<details>
    <summary><strong>Ответ</strong></summary>
    <p>Удалось.</p>
    <p>Логически можно продолжить рассуждения о том, что логические план (Analyzed Logical Plans) отличается от плана <code>taxi_cached_df</code> - выбирается только одна колонка. Фильтрация выполняется по <code>passenger_count</code>, который отсутствует в проекции (список колонок на выборку). Spark воспользуется правилом <code>ResolveMissingReferences</code> и добавит <code>passenger_count</code> к проекции, и тогда планы <code>taxi_fewer_columns_df</code> и <code>taxi_cached_df</code> станут идентичными. <i>Cache Manager</i> найдет закэшированный план и воспользуется им.</p>
</details>

## Выводы

### 1. Создавайте новый DataFrame при кэшировании другого DataFrame

Создание нового DataFrame при кэшировании позволит избежать проблем с кэшированием:

In [ ]:
taxi_cached_df.filter(taxi_df.passenger_count > 3) \
  .select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .explain()

In [ ]:
taxi_cached_df.select(taxi_df.trip_distance, taxi_df.passenger_count) \
  .filter(taxi_df.passenger_count > 7) \
  .explain()

In [ ]:
taxi_cached_df.select(taxi_df.trip_distance) \
  .filter(taxi_df.passenger_count > 3) \
  .explain()

### 2. Явно исключайте из кэша данные, которые больше не нужны

При помощи `df.unpersist()` можно явно убрать из кэша данные. Когда кэш заполняется, Spark начинает выбрасывать из кэша данных по стратегии LRU (Least Recent Used). Явный вызов `unpersist()` дает больше контроля над кэшем.

Плюс, чем меньше памяти занято кэшем, тем больше памяти Spark может использовать для выполнения работы (создание HashMap, и т.д.)

### 3. Кэшируйте только необходимое

Очень неэффектиным будет кэширование всего датасета, когда выборка идет лишь по нескольким полям. В этом случае, лучше:

1. Создать новый DataFrame, в котором выбраны только нужные колонки
1. Закешировать этот DataFrame
1. Выполнять выборку на базе закешированного DataFrame

In [ ]:
cached_taxi_df = taxi_df.cache()

In [ ]:
passenger_count_df = cached_taxi_df.select(cached_taxi_df.passenger_count)

In [ ]:
taxi_date_distance_df = cached_taxi_df.select(cached_taxi_df.tpep_pickup_datetime, cached_taxi_df.trip_distance)

**Так делать очень плохо**

Лушее решение: закэшировать только тот объем данных который реально будет использоваться

In [ ]:
cached_taxi_df.unpersist()

In [ ]:
cached_taxi_df = taxi_df.select(
    cached_taxi_df.passenger_count,
    cached_taxi_df.tpep_pickup_datetime,
    cached_taxi_df.trip_distance
  ) \
  .cache()

In [ ]:
passenger_count_df = cached_taxi_df.select(cached_taxi_df.passenger_count)

In [ ]:
taxi_date_distance_df = cached_taxi_df.select(cached_taxi_df.tpep_pickup_datetime, cached_taxi_df.trip_distance)

In [ ]:
cached_taxi_df.unpersist()

### 4. А может без кэширования быстрее?

## А может без кэширования быстрее?

Датафрейм `taxi_df` основан на базе parquet файла:

In [ ]:
taxi_df.explain()

Получить общее число строк:

In [ ]:
taxi_df.count()

Получить число строк, попадающих по фильтру:

In [ ]:
passengers_filter_taxi_df = taxi_df.filter(taxi_df.passenger_count > 3)

**(!!!)** План запроса показывает, что данные из parquet отфильтровываются уже на этапе чтения (**Pushed Filters**)

In [ ]:
passengers_filter_taxi_df.explain(mode="formatted")

In [ ]:
passengers_filter_taxi_df.count()

Положим `taxi_df` в кэш:

In [ ]:
taxi_cached_df = taxi_df.cache()

In [ ]:
taxi_cached_df.explain()

Первая опреация работает долго, т.к. нужно положить данные в кэш

In [ ]:
taxi_cached_df.count()

In [ ]:
taxi_cached_df.count()

Получить число строк из датафрейма в кэше, попадающих по фильтру:

In [ ]:
passengers_filter_taxi_cached_df = taxi_cached_df.filter(taxi_cached_df.passenger_count > 3)

План запроса показывает, что данные из parquet читаются целиком, а фильтрация происходит на более поздней страдии

In [ ]:
passengers_filter_taxi_cached_df.explain(mode="formatted")

In [ ]:
taxi_cached_df.unpersist()

### Выводы

1. Формат Parquet позволяет провести оптимизацию, при которой можно не читать ненужные данные из файла, если известно условие фильтрации
1. Кэширование вынуждает Spark прочитать весь parquet файл целиком
1. Кэширование отрезает Spark возможность применения оптимизации Pushed Filters

## Кэширование в SQL

Можно выполнять кэширование таблиц при работе со Spark через датафреймы.

In [ ]:
taxi_df.createOrReplaceTempView("taxi")

In [ ]:
spark.catalog.listTables()

In [ ]:
spark.sql("SELECT count(*) FROM taxi").show()

Инструкция `cache` позволяет закэшировать таблицу:

In [ ]:
spark.sql("cache table taxi")

In [ ]:
spark.sql("SELECT count(*) FROM taxi").show()

Инструкция `uncache` позволяет убрать таблицу из кэша:

In [ ]:
spark.sql("uncache table taxi")

Можно заметить, что в отличии от Spark DSL, кэширование в Spark SQL выполняется сразу, тогда как `cache()` метод в Spark DSL ленивая операция. Можно сделать операцию кэшировани в Spark SQL также ленивой:

In [ ]:
spark.sql("cache lazy table taxi")

Можно узнать находится ли таблица в кэше:

In [ ]:
spark.catalog.isCached("taxi")

In [ ]:
spark.sql("SELECT count(*) FROM taxi").show()

Можно очистить весь кэш при помощи `clearCache` и убрать все таблицы из кэша, но эта операция очень долгая, поэтому она является закомментированной

In [ ]:
# spark.catalog.clearCache()

## Кэширование плана

В ситуации, когда план становится очень длинным, есть риск, что случится StackOverflowError или драйвер будет тратить очень много времени на построение плана, или другие проблемы во время работы. План можно разбить на несколько частей, и обратиться к ним, когда возникнет необхоимость. Этого можно достичь при помощи механизма Checkpoint. При помощи Checkpoint можно разбить план и материализовать запрос. При обработке следующих трансформаций, Spark начнет создавать новый план.

In [ ]:
spark.sparkContext.setCheckpointDir("plan/checkpoint")

In [ ]:
small_taxi_df = taxi_df.limit(10)

In [ ]:
small_taxi_df = small_taxi_df.checkpoint()

In [ ]:
df = small_taxi_df.select(small_taxi_df.passenger_count)

In [ ]:
df.explain()

In [ ]:
df.show()

In [ ]:
df = taxi_df \
  .limit(10) \
  .select(small_taxi_df.passenger_count)

In [ ]:
df.explain()

In [ ]:
df.show()